In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0539,  0.0287,  0.1540, -0.1064, -0.0486,  0.0976,  0.3424, -0.3222,
          0.0243, -0.1563],
        [-0.0095, -0.0179,  0.1302, -0.1228,  0.0093,  0.1753,  0.1790, -0.1774,
          0.0051, -0.1918]], grad_fn=<AddmmBackward0>)

## 1 继承Module类来构造模型

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.1693, -0.2198,  0.1756, -0.2204, -0.0962,  0.0329, -0.2671, -0.4018,
          0.2565,  0.0114],
        [-0.2469, -0.0758,  0.2663, -0.0730, -0.0912,  0.1079, -0.0872, -0.1593,
          0.2410,  0.0986]], grad_fn=<AddmmBackward0>)

## 2 Module的子类

### 2.1 Sequential类

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # 这里，`block`是`Module`子类的一个实例。我们把它保存在'Module'类的成员变量
            # `_modules` 中。`block`的类型是OrderedDict。
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1427,  0.1705,  0.2294, -0.0717,  0.1826,  0.0888, -0.0362,  0.0734,
         -0.5342, -0.2601],
        [-0.0887,  0.1842,  0.2278,  0.0366,  0.1085,  0.0302, -0.0505,  0.0420,
         -0.3528, -0.2607]], grad_fn=<AddmmBackward0>)

### 2.2 ModuleList类

### 2.3 ModuleDict类

## 3 构造复杂的模型

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变。
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及`relu`和`dot`函数。
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1686, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2461, grad_fn=<SumBackward0>)